In [1]:
import os
import gdown
import winsound
import numpy as np
import onnxruntime as ort
import albumentations as A
from PIL import Image
from tqdm import tqdm
from scipy.special import softmax
from albumentations.pytorch import ToTensorV2

In [3]:
url = "https://drive.google.com/uc?id=1-iWtl9VHyDIypQnsRAuNzlkLfxddVXDi"
gdown.download(url)

Downloading...
From: https://drive.google.com/uc?id=1-iWtl9VHyDIypQnsRAuNzlkLfxddVXDi
To: d:\Projects\Project Programming\Python\Indonesia AI\RnD - AI for Disaster Management\ai-for-disaster-management\utils\saved_model.onnx
100%|██████████| 90.4M/90.4M [03:18<00:00, 456kB/s]


'saved_model.onnx'

In [2]:
model_path = "./saved_model.onnx"
device_inference = "cpu"
classes = ['street', 'mountain', 'sea', 'buildings', 'forest', 'glacier']

In [3]:
def predict_image(image, ort_session):
    input_onnx = ort_session.get_inputs()[0].name
    outputs = ort_session.run(
        None,
        {input_onnx: image},
    )
    return outputs

def preprocessing_image(image_path):
    image = Image.open(image_path)
    image_augmentation = A.Compose([
            A.Resize(224, 224),
            A.Normalize(),
            ToTensorV2()
    ])
    image = image_augmentation(image=np.array(image))["image"]
    image = np.expand_dims(image, axis=0)
    return image

def postprocessing_output(output):
    predict = softmax(output[0][0])
    predict = np.argmax(predict)
    predict = classes[predict]
    return predict

def play_sound(duration, freq):
    winsound.Beep(freq, duration)

In [4]:
ort_session = (ort.InferenceSession(model_path, providers=["CPUExecutionProvider" 
                                                           if device_inference == "cpu" 
                                                           else "CUDAExecutionProvider"])
)

In [5]:
path_image_dir = "./dataset/day/"
all_images = os.listdir(path_image_dir)

In [6]:
for image in tqdm(all_images):
    try:
        predict = preprocessing_image(os.path.join(path_image_dir, image))
        predict = predict_image(predict, ort_session)
        predict = postprocessing_output(predict)
        if predict == "forest": continue
        os.remove(os.path.join(path_image_dir, image))
    except:
        os.remove(os.path.join(path_image_dir, image))
        continue

100%|██████████| 6361/6361 [17:10<00:00,  6.18it/s]


In [7]:
play_sound(2000, 500)